<a href="https://colab.research.google.com/github/cheikh19/cheikh19/blob/main/exempleZola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=d7e05c822da5c04dcabb4a32b507fc92ed380dc8e9fe18c1f05f8dcbdff153d5
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Modele RF').getOrCreate()



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Modele RF').getOrCreate()

# Chargez le fichier CSV dans un dataframe Spark
diabetes = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("/content/diabetes.csv")

In [131]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.ml.classification import  LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel


In [67]:
#lire le fichier csv
diabetes.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|      0|25.6|                   0.294| 28|      0|


In [68]:
# vérifier les dimensions du df
print ( diabetes . count (), ':' , len ( diabetes . columns ))

2000 : 9


In [69]:
for col in diabetes.columns:
  print(col, "\t", "with null values: ", diabetes.filter(diabetes[col].isNull()).count())



Pregnancies 	 with null values:  0
Glucose 	 with null values:  0
BloodPressure 	 with null values:  0
SkinThickness 	 with null values:  0
Insulin 	 with null values:  0
BMI 	 with null values:  0
DiabetesPedigreeFunction 	 with null values:  0
Age 	 with null values:  0
Outcome 	 with null values:  0


Notre *dataset* ne contient aucune valeur manquante.

In [70]:
# créer une fonction pour compter le nombre de valeurs 0 et leur pourcentage par colonne
for  col  in  diabetes.columns:
    num_zeros  =  diabetes . filter( diabetes [ col ] ==  0 ). count ()
    total_rows  =  diabetes . count ()
    percentage  = ( num_zeros  /  total_rows ) *  100
    print ( "{} : {} ({:.2f}%)" . format ( col , num_zeros , percentage ))

Pregnancies : 301 (15.05%)
Glucose : 13 (0.65%)
BloodPressure : 90 (4.50%)
SkinThickness : 573 (28.65%)
Insulin : 956 (47.80%)
BMI : 28 (1.40%)
DiabetesPedigreeFunction : 0 (0.00%)
Age : 0 (0.00%)
Outcome : 1316 (65.80%)


In [71]:
# Convertir les colonnes en vecteur
featureCols = diabetes.columns[:-1]
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
outputCol = assembler.transform(diabetes).select(col("features"), col("Outcome").alias("label"))

TypeError: ignored

In [72]:
# calculer la corrélation entre la variable de réponse et les autres variables
for  col in  diabetes.columns :
  #print ( 'La corrélation de {} avec la variable issue est {}.' . format ( col , diabetes . stat . corr ( 'Outcome' , col )))
  print(f'La corrélation de {col} avec la variable issue est {diabetes.stat.corr("Outcome", col)}.')

La corrélation de Pregnancies avec la variable issue est 0.22443699263363961.
La corrélation de Glucose avec la variable issue est 0.4584213024234631.
La corrélation de BloodPressure avec la variable issue est 0.07595808358896636.
La corrélation de SkinThickness avec la variable issue est 0.07604024696581364.
La corrélation de Insulin avec la variable issue est 0.12092362183050201.
La corrélation de BMI avec la variable issue est 0.27672553877003214.
La corrélation de DiabetesPedigreeFunction avec la variable issue est 0.1554590791569403.
La corrélation de Age avec la variable issue est 0.23650924717620253.
La corrélation de Outcome avec la variable issue est 1.0.


"""Les résultats de corrélation indiquant que certains facteurs peuvent avoir un impact significatif sur le résultat de la régression logistique.
* Les variables `Glucose` et `BMI` ont les **plus fortes corrélations** avec le résultat, ce qui signifie qu'elles sont les plus susceptibles d'avoir un impact sur le résultat de la régression logistique.
* Les variables `Pregnancies`, `BloodPressure`, `SkinThickness`, `Insulin` et `DiabetesPedigreeFunction` ont des **corrélations plus faibles** avec le résultat, ce qui signifie qu'elles sont moins susceptibles d'avoir un impact sur le résultat de la régression logistique.
* La variable `Age` a une corrélation **modérée** avec le résultat, ce qui signifie qu'elle peut avoir un impact modéré sur le résultat de la régression logistique

In [73]:
# Convertir les colonnes en vecteur
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
outputCol  =  assembler.transform (diabetes)

In [74]:
# vérifier si la colonne features est rajoutée au dataframe
outputCol.printSchema ()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [75]:
# afficher les données (3 lignes)
outputCol.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[0.0,84.0,82.0,31...|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|(8,[1,5,6,7],[145...|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[0.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          0|    173|           

"""## Entrainer le modèle"""

In [76]:
# sélectionner les colonnes d'intérêt
diabetes =  outputCol.select ("features", "Result")
diabetes.show ( )

AnalysisException: ignored

In [86]:
# Diviser les données en ensembles d'entraînement et de test
train_diabete, test_diabete = diabetes.randomSplit([0.7, 0.3], seed=42)

In [96]:
# split to training (70%) and test (30%)
train, test = test_diabete.randomSplit([0.7, 0.3])

In [97]:
# Créer le modèle
rf = RandomForestClassifier(numTrees=10, maxDepth=5, seed=42)

In [110]:
# selectionner les colonnes d'interer
final_df = outputCol.select('features', 'Outcome')
final_df.show(2)

+--------------------+-------+
|            features|Outcome|
+--------------------+-------+
|[2.0,138.0,62.0,3...|      1|
|[0.0,84.0,82.0,31...|      0|
+--------------------+-------+
only showing top 2 rows



In [113]:
# split to training (70%) and test (30%)
train, test = final_df.randomSplit([0.7, 0.3])

In [115]:
# Créer le modèle
models  =  LogisticRegression ( labelCol = "Outcome" )

In [116]:
# entrainer le modèle
model = models.fit(train)

In [118]:
# afficher le sommaire
summary = model.summary

In [119]:
## les prédictions
summary.predictions.show()
summary.predictions.describe().show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(8,[0,1,6,7],[2.0...|    0.0|[4.93115348297545...|[0.99283355618866...|       0.0|
|(8,[0,1,6,7],[2.0...|    0.0|[4.40908067509732...|[0.98797988302053...|       0.0|
|(8,[0,1,6,7],[2.0...|    0.0|[4.40908067509732...|[0.98797988302053...|       0.0|
|(8,[0,1,6,7],[3.0...|    0.0|[4.53777055731574...|[0.98941599072233...|       0.0|
|(8,[0,1,6,7],[3.0...|    0.0|[4.53777055731574...|[0.98941599072233...|       0.0|
|(8,[0,1,6,7],[3.0...|    0.0|[4.53777055731574...|[0.98941599072233...|       0.0|
|(8,[0,1,6,7],[3.0...|    0.0|[4.53777055731574...|[0.98941599072233...|       0.0|
|(8,[0,1,6,7],[6.0...|    0.0|[2.94850689574716...|[0.95019287270607...|       0.0|
|(8,[0,1,6,7],[7.0...|    0.0|[3.05293810871259...|[0.95490920369670...|    


"""## Évaluation du modèle
`BinaryClassificationEvaluator` de pysparkML est un outil d'évaluation qui permet de **mesurer la performance d'un modèle de classification binaire**.
Il fournit des métriques telles que la `précision`, le `rappel`, l'`aire sous la courbe ROC (AUC) : par défaut` et la `précision-rappel`. Ces métriques peuvent être utilisées pour comparer les **performances** des modèles et ainsi déterminer le meilleur modèle à utiliser.
"""

In [121]:
# alimentez les données de test dans le modèle et évaluez-les
predictions = model.evaluate(test)

In [123]:
# Voir les prédictions
predictions.predictions.show(10)

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(8,[0,1,6,7],[2.0...|      0|[4.93115348297545...|[0.99283355618866...|       0.0|
|(8,[0,1,6,7],[3.0...|      0|[4.53777055731574...|[0.98941599072233...|       0.0|
|(8,[0,1,6,7],[10....|      1|[2.30698787780216...|[0.90945412082235...|       0.0|
|(8,[1,5,6,7],[99....|      0|[2.19437484591270...|[0.89974323165732...|       0.0|
|(8,[1,5,6,7],[119...|      1|[1.00769789308943...|[0.73256938160046...|       0.0|
|(8,[1,5,6,7],[141...|      1|[-0.6703641093810...|[0.33841531541021...|       1.0|
|(8,[1,5,6,7],[145...|      1|[-1.3943524140496...|[0.19871382770406...|       1.0|
|(8,[1,5,6,7],[145...|      1|[-1.3943524140496...|[0.19871382770406...|       1.0|
|(8,[1,5,6,7],[145...|      1|[-1.3943524140496...|[0.19871382770406...|    

Dans la plupart des cas (sur les 10 lignes affichées) le modele de régressin predit bien les resultats

In [124]:
from pyspark.ml import evaluation

In [125]:
#Evaluer le modele
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='Outcome')
evaluator.evaluate(model.transform(test))

0.8366023166023154

les résultats ont montré que le modéle de regression a une précision de 83%, ce qui est assez bon . Ce qui signifie que le modéle est capable de prédire avec une précision élevée si un individu aura un résultat positif (malade) ou négatif (diabétique).

# Sauvegarder le modele

""" On sauvegarde un modèle de machine learning afin de pouvoir le réutiliser à l'avenir. En effet, une fois que le modèle a été entraîné et qu'il fonctionne de manière optimale, il est important de le sauvegarder afin de pouvoir le réutiliser pour de nouvelles prédictions. Cela permettra d'économiser du temps et des ressources, car il n'est pas nécessaire de recycler le modèle pour chaque nouvelle prédiction.
"""




In [129]:
model.save("LogReg_model")

"""## Réutiliser le modèle sauvegardé"""

In [132]:
model = LogisticRegressionModel.load('LogReg_model')
